梯度下降--让模型逼近最小偏差
# 梯度下降的作用及分类
梯度下降法是一个最优算法，常用于机器学习和人工智能中递归性地逼近最小偏差模型，梯度下降的方向也就是用负梯度方向为搜索方向，沿着梯度下降的方向求解极小值。

在训练过程中，每次的正向传播后都会得到输出值与真实值的损失值，这个损失值越小，代表模型越好，于是梯度下降的算法就用在这里，帮助寻找最小的那个损失值，从而可以反推出对应的学习参数b和w，达到优化模型的效果。

> 常用的梯度下降方法可以分为：**批量梯度下降**、**随机梯度下降**和**小批量梯度下降**。

- 批量梯度下降：